# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 6 2023 8:54AM,254436,10,0086284603,ISDIN Corporation,Released
1,Jan 6 2023 8:54AM,254436,10,0086284601,ISDIN Corporation,Released
2,Jan 6 2023 8:54AM,254436,10,0086284600,ISDIN Corporation,Released
3,Jan 6 2023 8:54AM,254436,10,0086284602,ISDIN Corporation,Released
4,Jan 6 2023 8:54AM,254436,10,0086284609,ISDIN Corporation,Released
5,Jan 6 2023 8:54AM,254436,10,0086284614,ISDIN Corporation,Released
6,Jan 6 2023 8:54AM,254436,10,0086284617,ISDIN Corporation,Released
7,Jan 6 2023 8:54AM,254436,10,0086284618,ISDIN Corporation,Released
8,Jan 6 2023 8:54AM,254436,10,0086284616,ISDIN Corporation,Released
9,Jan 6 2023 8:54AM,254436,10,0086284615,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
49,254433,Released,22
50,254434,Released,1
51,254436,Released,60
52,254437,Released,23
53,254438,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254433,NaN,NaN,22.0
254434,NaN,NaN,1.0
254436,NaN,NaN,60.0
254437,NaN,NaN,23.0
254438,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254331,0.0,0.0,1.0
254346,0.0,0.0,1.0
254347,0.0,0.0,1.0
254348,0.0,1.0,0.0
254351,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254331,0,0,1
254346,0,0,1
254347,0,0,1
254348,0,1,0
254351,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254331,0,0,1
1,254346,0,0,1
2,254347,0,0,1
3,254348,0,1,0
4,254351,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254331,,,1
1,254346,,,1
2,254347,,,1
3,254348,,1,
4,254351,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 6 2023 8:54AM,254436,10,ISDIN Corporation
60,Jan 6 2023 8:50AM,254438,10,Emerginnova
62,Jan 6 2023 8:50AM,254437,10,ISDIN Corporation
85,Jan 6 2023 8:46AM,254433,12,Hush Hush
107,Jan 6 2023 8:45AM,254434,12,Hush Hush
108,Jan 6 2023 8:44AM,254432,10,ISDIN Corporation
117,Jan 6 2023 8:42AM,254431,10,"Nextsource Biotechnology, LLC"
119,Jan 6 2023 8:42AM,254430,20,"Adare Pharmaceuticals, Inc."
120,Jan 6 2023 8:38AM,254429,19,"Granules Pharmaceuticals, Inc."
121,Jan 6 2023 8:31AM,254427,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 6 2023 8:54AM,254436,10,ISDIN Corporation,,,60
1,Jan 6 2023 8:50AM,254438,10,Emerginnova,,,2
2,Jan 6 2023 8:50AM,254437,10,ISDIN Corporation,,,23
3,Jan 6 2023 8:46AM,254433,12,Hush Hush,,,22
4,Jan 6 2023 8:45AM,254434,12,Hush Hush,,,1
5,Jan 6 2023 8:44AM,254432,10,ISDIN Corporation,,,9
6,Jan 6 2023 8:42AM,254431,10,"Nextsource Biotechnology, LLC",,,2
7,Jan 6 2023 8:42AM,254430,20,"Adare Pharmaceuticals, Inc.",,,1
8,Jan 6 2023 8:38AM,254429,19,"Granules Pharmaceuticals, Inc.",,,1
9,Jan 6 2023 8:31AM,254427,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:54AM,254436,10,ISDIN Corporation,60,,
1,Jan 6 2023 8:50AM,254438,10,Emerginnova,2,,
2,Jan 6 2023 8:50AM,254437,10,ISDIN Corporation,23,,
3,Jan 6 2023 8:46AM,254433,12,Hush Hush,22,,
4,Jan 6 2023 8:45AM,254434,12,Hush Hush,1,,
5,Jan 6 2023 8:44AM,254432,10,ISDIN Corporation,9,,
6,Jan 6 2023 8:42AM,254431,10,"Nextsource Biotechnology, LLC",2,,
7,Jan 6 2023 8:42AM,254430,20,"Adare Pharmaceuticals, Inc.",1,,
8,Jan 6 2023 8:38AM,254429,19,"Granules Pharmaceuticals, Inc.",1,,
9,Jan 6 2023 8:31AM,254427,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:54AM,254436,10,ISDIN Corporation,60,,
1,Jan 6 2023 8:50AM,254438,10,Emerginnova,2,,
2,Jan 6 2023 8:50AM,254437,10,ISDIN Corporation,23,,
3,Jan 6 2023 8:46AM,254433,12,Hush Hush,22,,
4,Jan 6 2023 8:45AM,254434,12,Hush Hush,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:54AM,254436,10,ISDIN Corporation,60.0,NaN,NaN
1,Jan 6 2023 8:50AM,254438,10,Emerginnova,2.0,NaN,NaN
2,Jan 6 2023 8:50AM,254437,10,ISDIN Corporation,23.0,NaN,NaN
3,Jan 6 2023 8:46AM,254433,12,Hush Hush,22.0,NaN,NaN
4,Jan 6 2023 8:45AM,254434,12,Hush Hush,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 6 2023 8:54AM,254436,10,ISDIN Corporation,60.0,0.0,0.0
1,Jan 6 2023 8:50AM,254438,10,Emerginnova,2.0,0.0,0.0
2,Jan 6 2023 8:50AM,254437,10,ISDIN Corporation,23.0,0.0,0.0
3,Jan 6 2023 8:46AM,254433,12,Hush Hush,22.0,0.0,0.0
4,Jan 6 2023 8:45AM,254434,12,Hush Hush,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4579201,336.0,13.0,10.0
12,508867,23.0,0.0,0.0
15,508757,5.0,0.0,0.0
16,1272083,5.0,1.0,0.0
18,254420,1.0,0.0,0.0
19,2798448,25.0,0.0,4.0
20,763182,3.0,0.0,0.0
21,2035017,7.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4579201,336.0,13.0,10.0
1,12,508867,23.0,0.0,0.0
2,15,508757,5.0,0.0,0.0
3,16,1272083,5.0,1.0,0.0
4,18,254420,1.0,0.0,0.0
5,19,2798448,25.0,0.0,4.0
6,20,763182,3.0,0.0,0.0
7,21,2035017,7.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,336.0,13.0,10.0
1,12,23.0,0.0,0.0
2,15,5.0,0.0,0.0
3,16,5.0,1.0,0.0
4,18,1.0,0.0,0.0
5,19,25.0,0.0,4.0
6,20,3.0,0.0,0.0
7,21,7.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,336.0
1,12,Released,23.0
2,15,Released,5.0
3,16,Released,5.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
Executing,13.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
Released,336.0,23.0,5.0,5.0,1.0,25.0,3.0,7.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,13.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,Released,336.0,23.0,5.0,5.0,1.0,25.0,3.0,7.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
1,Executing,13.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,Released,336.0,23.0,5.0,5.0,1.0,25.0,3.0,7.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()